# Comparison star generator

In [ ]:
import re

import requests

from astropy.coordinates import SkyCoord

from stellarphot.visualization.comparison_functions import read_file, ComparisonViewer


## Change the settings in this cell as needed

In [ ]:
directory_with_images = '..'

# 👉 This could be any image
sample_image_for_finding_stars = 'TIC_237205154.01-S001-R013-C001-rp.fit'

# 👉 You will be able to use this name later to pull out the data you want
object_of_interest = ''

# 👉 Enter the coordinates of your object 
ra_object = '19:34:19.97'
dec_object = '76:39:39.91'

bright_mag_limit = 9
dim_mag_limit = 16
Cmag = 10

# 👉 File to save apertures in
aperture_output_file = 'aperture_locations.csv'

In [ ]:
try:
    coordinate = SkyCoord.from_name(object_of_interest)
except:
    coordinate = SkyCoord(ra=ra_object, dec=dec_object, unit=("hour", "degree"))

In [ ]:
comp_viewer = ComparisonViewer(sample_image_for_finding_stars,
                           directory=directory_with_images,
                           target_mag=Cmag,
                           bright_mag_limit=bright_mag_limit,
                           dim_mag_limit=dim_mag_limit,
                           targets_from_file=None,
                           object_coordinate=coordinate,
                             aperture_output_file=aperture_output_file
                          )
comp_viewer.box